```

| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |  열차 하나 
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
                            .
                            .
                            .
                            .
                            .
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |  열차 백 

```

전체 구간에 대해 유전 알고리즘을 적용시킴니다.

In [2]:
# import 및 변수정의 구간
import random
import math
import re
import operator 
from random import randint

# 리스트 크기
listN = 8

# 염색체 해집단 수
N = 30

# 역 구간
Station_section = 11

# 하루에 운행하는 열차 수
One_day_train = 100

#교차율
Pc = 0.7

#변이율 역수 관계
Pm = 100

#마지막 세대수 
largest_generation = 300

#보존할 상위값
save_value = 5


In [3]:
# 01010로 이루어진 리스트를 10진수로 디코딩하는 함수

def decoding(X, Y=0):
    """
    input   : 0100로 이루어진 리스트
    return : 10진수 정수
    """
    for i, x in enumerate(X[1:]):
        Y += x* math.pow(2, listN -i -2)
    #처음 값이 0이면 음수로 return
    if X[0] == 0: return -Y
    return Y

In [4]:
#파일을 2차원 배열로 바꿔주기

def Readtxt(fname):
    """
    input  : 파일들
    return : 2차원 배열 
    """
    w = []
    with open(fname, 'r') as f :
        for line in f: #for each line in the file,
            w.append(line.split('\t'))
    return w

In [5]:
# 바뀌야됨
#def ScoreFunction(X, 구간,):

def ScoreFunction(X):
    """
    X에 속도 리스트를 넣어서 평가
    """
    AirportNum = Readtxt('data/Number-of-train-people.txt')
    AirportTime = Readtxt('data/Time-of-airport-train.txt')
    AtherTime = Readtxt('data/Time-of-ather-train.txt')
    sp = decoding(X)
    return (777 - (6 * sp)) * 8

In [6]:
#초기값
speedlist = [[[[random.choice([0,1]) for i in range(listN)]for j in range(Station_section)]for k in range(One_day_train)]for z in range(N)]



In [7]:
print(len(speedlist), len(speedlist[0]), len(speedlist[0][0]),len(speedlist[0][0][0]))

30 100 11 8


In [12]:
#하루 열차의 총합 구하기
def Make_Score_per_all(speedlist):
    #열차마다 점수                             #30개의 경쟁 100개의 열차
    Score_per_train = [[0 for i in range(One_day_train)]for j in range(N)]

    # 30개의 경쟁
    for k in range(N):
        # 100대의 열차
        for i in range(One_day_train):
            # 11개의 역
            for j in range(Station_section):
                #각 열차에 대해      += 11개의 구간의 점수 합                   ([30경쟁][100대열차] +=  [30경쟁][100대열차][11개의 역])
                Score_per_train[k][i] += ScoreFunction(speedlist[k][i][j])
    
    # 하루마다의 점수 
    Score_per_all = [0 for i in range(N)]

    # 30개의 경쟁            
    for i in range(N):
        # 100대의 열차
        for j in range(One_day_train):
             Score_per_all[i] += Score_per_train[i][j]
                
    return Score_per_all

In [13]:
test_Score_per_all = Make_Score_per_all(speedlist)
test_Score_per_all

[6979488.0,
 6923424.0,
 6898032.0,
 6864144.0,
 6827424.0,
 6817632.0,
 6896304.0,
 6789792.0,
 6745344.0,
 6809088.0,
 6846000.0,
 6940896.0,
 6832608.0,
 6943200.0,
 6893280.0,
 7112976.0,
 6787392.0,
 7060848.0,
 6974928.0,
 6895968.0,
 6929040.0,
 6842352.0,
 6908784.0,
 6908064.0,
 6564960.0,
 6813072.0,
 6836352.0,
 6893424.0,
 6837072.0,
 6872112.0]

In [64]:
print(len(speedlist), len(speedlist[0]), len(speedlist[0][0]), len(speedlist[0][0][0]),"총개수 : " ,len(speedlist) * len(speedlist[0]) * len(speedlist[0][0]) * len(speedlist[0][0][0])
)

print("-"*100)

#하루마다의 점수 (이걸로 경쟁)
print(test_Score_per_all[1])
#열차마다의 점수
#print(Score_per_train[1][2])
#구간마다의 점수
print(ScoreFunction(speedlist[1][2][1]))


30 100 11 8 총개수 :  264000
----------------------------------------------------------------------------------------------------
6826848.0
6120.0


In [7]:
def Score_assessment_group(Score_per_all):
    # 적합도 비율 찾기
    speedlist_evaluation = [[Score_per_all[i], 0, i] for i in range(N)]

    Sum_of_fitness = 0
    for i in range(N): 
        Sum_of_fitness += Score_per_all[i]
    for i in range(N):
        if i == 0:
            speedlist_evaluation[i][1] = speedlist_evaluation[i][0]/Sum_of_fitness
        else:
            speedlist_evaluation[i][1] = speedlist_evaluation[i-1][1]  + speedlist_evaluation[i][0]/Sum_of_fitness
    
    return speedlist_evaluation


In [66]:
#평가한 값 
Group_assessment_value = Score_assessment_group(Score_per_all)

#                                         정렬   점수값이 높은 순으로
sortand_Group_assessmend = sorted(Group_assessment_value, key=operator.itemgetter(0),reverse=True)
#상위 save_value 개값        
save_assessmend_value = sortand_Group_assessmend[0:save_value]
# Score_per_all에서 몇번 배열이 상위 5개인지만 
# 리스트 해쉬값만
save_assessmend_num = [save_assessmend_value[i][2] for i in range(save_value)]

print(save_assessmend_num)

print(Score_per_all[save_assessmend_num[0]])

Group_assessment_value

[11, 19, 8, 27, 7]
7103472.0


[[6890016.0, 0.03344915549865555, 0],
 [6826848.0, 0.06659164752156024, 1],
 [6682800.0, 0.09903482564573518, 2],
 [6813072.0, 0.13211043893190696, 3],
 [6787728.0, 0.1650630139846468, 4],
 [6849072.0, 0.1983133974887617, 5],
 [6825408.0, 0.23144889870294866, 6],
 [6931632.0, 0.26510008857354644, 7],
 [7021968.0, 0.29918983517770287, 8],
 [6857040.0, 0.3324789011567225, 9],
 [6862848.0, 0.3657961633975703, 10],
 [7103472.0, 0.40028158977514994, 11],
 [6861936.0, 0.43359442450380986, 12],
 [6753888.0, 0.46638271555168315, 13],
 [6791184.0, 0.49935206854534553, 14],
 [6667008.0, 0.5317185808005828, 15],
 [6824592.0, 0.5648501205564963, 16],
 [6766320.0, 0.5976987655862993, 17],
 [6908448.0, 0.6312374034365418, 18],
 [7098576.0, 0.6656990610644812, 19],
 [6893904.0, 0.6991670917466746, 20],
 [6915264.0, 0.7327388194248475, 21],
 [6928800.0, 0.7663762607049671, 22],
 [6690384.0, 0.7988562570883887, 23],
 [6906192.0, 0.8323839426716401, 24],
 [6930576.0, 0.8660300059491782, 25],
 [6918672.0

In [8]:
def roulettes(Group_assessment_value):
    roulette = random.random()
    for i in range(N):
        if i == 0:
            if 0 < roulette < Group_assessment_value[i][1]:
                break
        if Group_assessment_value[i-1][1] < roulette < Group_assessment_value[i][1]:
            break
    return i

In [68]:
Selected_list_in_roulette = []

#룰렛에 선택된 리스트들 마지막 5개는 상위5개만 
for i in range(N-5):
    Selected_list_in_roulette.append(roulettes(Group_assessment_value))

Selected_list_in_roulette += save_assessmend_num


Selected_list_in_roulette

[26,
 4,
 29,
 22,
 13,
 8,
 3,
 1,
 14,
 3,
 13,
 5,
 22,
 20,
 15,
 21,
 6,
 18,
 19,
 23,
 26,
 26,
 14,
 0,
 26,
 11,
 19,
 8,
 27,
 7]

In [69]:
len(Selected_list_in_roulette)

30

In [9]:
#부모 변경
def Change_parent(speedlist, Selected_list_in_roulette):
    W = [[[0 for j in range(Station_section)]for k in range(One_day_train)]for z in range(N)]
    # 교차함수 및 변이함수 (자식만들기)
    for i in range(int(N/2)):
        for j in range(One_day_train):
            for k in range(Station_section):
                #교차
                selection = random.random()
                ###############################################################
                if selection < Pc:
                    rand = random.choice([i+1 for i in range(listN-1)]) # 0~ 이구간, 이구간 ~ 끝 교차 , 이구간 = 1~리스트 크기 -1 
                    # W(교차된 자식) = Selected_list_in_roulette(선택된 리스트에서) rand구간으로 나눈 값
                    W[i*2][j][k]     = speedlist[Selected_list_in_roulette[i*2]][j][k][0:rand]         + speedlist[Selected_list_in_roulette[i*2+1]][j][k][rand:]
                    W[i*2+1][j][k] = speedlist[Selected_list_in_roulette[i*2+1]][j][k][0:rand] + speedlist[Selected_list_in_roulette[i*2]][j][k][rand:]
                else:
                    W[i*2][j][k]     = speedlist[Selected_list_in_roulette[i*2]][j][k]
                    W[i*2+1][j][k] = speedlist[Selected_list_in_roulette[i*2+1]][j][k]
                #변의
                for z in range(2):
                    # Pm * 100 분의 1의 확률 
                    if randint(0, 100 * Pm) == 0:
                        #selection 부분을 변의 1 -> 0  0 -> 1
                        if W[i*2+z][j][k][rand] == 0:
                            W[i*2+z][j][k][rand] = 1
                        else:
                            W[i*2+z][j][k][rand] = 0
    return W

In [71]:
test_new_lest = Change_parent(speedlist, Selected_list_in_roulette)

In [72]:
print(len(test_new_lest), len(test_new_lest[0]), len(test_new_lest[0][0]),len(test_new_lest[0][0][0]))

30 100 11 8


In [73]:
test_Score_per_all = Make_Score_per_all(test_new_lest)
test_Score_per_all

[6982512.0,
 6774096.0,
 6850704.0,
 6955296.0,
 6783840.0,
 7007664.0,
 6838224.0,
 6784512.0,
 6819696.0,
 6867408.0,
 6783600.0,
 6900672.0,
 6916560.0,
 6869952.0,
 6664176.0,
 6861360.0,
 6866016.0,
 6977568.0,
 7089120.0,
 6697344.0,
 6918672.0,
 6918672.0,
 6768432.0,
 6915264.0,
 6952416.0,
 7106016.0,
 7094160.0,
 7005360.0,
 6968160.0,
 6900720.0]

In [10]:
#세대 반복
def Evolution(speedlist, num_generation):
    while 1: 
        if num_generation >= largest_generation:
            break

        #전체 값으로 해서 
        Score_per_all = Make_Score_per_all(speedlist)

        #평가한 값 
        Group_assessment_value = Score_assessment_group(Score_per_all)

        #                                         정렬   점수값이 높은 순으로
        sortand_Group_assessmend = sorted(Group_assessment_value, key=operator.itemgetter(0),reverse=True)
        #상위 save_value 개값        
        save_assessmend_value = sortand_Group_assessmend[0:save_value]
        # Score_per_all에서 몇번 배열이 상위 5개인지만 
        # 리스트 해쉬값만
        save_assessmend_num = [save_assessmend_value[i][2] for i in range(save_value)]


        Selected_list_in_roulette = []

        #룰렛에 선택된 리스트들 마지막 5개는 상위5개만 
        for i in range(N-5):
            Selected_list_in_roulette.append(roulettes(Group_assessment_value))

        Selected_list_in_roulette += save_assessmend_num
        
        #룰렛 랜덤하게  근데이거하면 오류남 왜지?
        #Selected_list_in_roulette = random.shuffle(Selected_list_in_roulette)
        # ------------------------------------------------------------------------------------------------------------------------------------------------------------
        
        New_list = Change_parent(speedlist, Selected_list_in_roulette)
        
        New_Score_per_all = Make_Score_per_all(New_list)
        
        print("-" * 15)    
        print(num_generation,"번째를 합니다.")
        print("-" * 15)
        for i, nl in enumerate(New_Score_per_all):
            print(i, nl)
        
        speedlist = New_list
        num_generation += 1

    return New_list

In [11]:
test_list = []

test_list = Evolution(speedlist, 0)

---------------
0 번째를 합니다.
---------------
0 6758784.0
1 6758784.0
2 7038048.0
3 6746448.0
4 6902736.0
5 6928896.0
6 6731856.0
7 6903360.0
8 6907344.0
9 6872256.0
10 6671328.0
11 7155120.0
12 6877248.0
13 6927696.0
14 6901872.0
15 6815376.0
16 7078608.0
17 6868464.0
18 6851184.0
19 7011552.0
20 6844320.0
21 6883824.0
22 6791664.0
23 6829440.0
24 6805392.0
25 7033392.0
26 7038768.0
27 6977520.0
28 6947472.0
29 6935328.0
---------------
1 번째를 합니다.
---------------
0 6704592.0
1 6765408.0
2 6780384.0
3 7109424.0
4 6860928.0
5 6916752.0
6 7130640.0
7 7057872.0
8 6739728.0
9 6905664.0
10 7033392.0
11 7033392.0
12 6902112.0
13 7019952.0
14 6730032.0
15 6762912.0
16 6985824.0
17 6799104.0
18 6831744.0
19 6846672.0
20 6844320.0
21 6844320.0
22 6750384.0
23 6865200.0
24 6988320.0
25 7158528.0
26 7017360.0
27 7021296.0
28 7060224.0
29 7022064.0
---------------
2 번째를 합니다.
---------------
0 6844272.0
1 6844272.0
2 7102848.0
3 7049856.0
4 6811968.0
5 7042224.0
6 6844272.0
7 6844272.0
8 7034352.0
9 6

---------------
20 번째를 합니다.
---------------
0 7197600.0
1 7231104.0
2 7154112.0
3 7217760.0
4 7196304.0
5 7232496.0
6 7218000.0
7 7191600.0
8 7174224.0
9 7214832.0
10 7196400.0
11 7210848.0
12 7191216.0
13 7162992.0
14 7228080.0
15 7103328.0
16 7226304.0
17 7219728.0
18 7215120.0
19 7238448.0
20 7179120.0
21 7194432.0
22 7159584.0
23 7170384.0
24 7217136.0
25 7236048.0
26 7216320.0
27 7240080.0
28 7216800.0
29 7228704.0
---------------
21 번째를 합니다.
---------------
0 7210656.0
1 7225824.0
2 7251888.0
3 7207920.0
4 7224048.0
5 7243104.0
6 7216464.0
7 7190688.0
8 7168320.0
9 7182096.0
10 7179120.0
11 7202112.0
12 7226496.0
13 7219680.0
14 7242576.0
15 7228368.0
16 7242048.0
17 7213200.0
18 7219440.0
19 7187712.0
20 7206240.0
21 7222656.0
22 7221072.0
23 7201632.0
24 7211808.0
25 7246032.0
26 7246608.0
27 7227888.0
28 7235280.0
29 7228320.0
---------------
22 번째를 합니다.
---------------
0 7218048.0
1 7207632.0
2 7211808.0
3 7219392.0
4 7194672.0
5 7215456.0
6 7212096.0
7 7240032.0
8 7248912.0


---------------
40 번째를 합니다.
---------------
0 7360464.0
1 7327056.0
2 7336272.0
3 7347360.0
4 7321056.0
5 7349712.0
6 7336848.0
7 7351056.0
8 7339680.0
9 7339872.0
10 7336752.0
11 7333920.0
12 7339968.0
13 7339968.0
14 7342032.0
15 7342032.0
16 7318944.0
17 7318032.0
18 7297056.0
19 7335840.0
20 7305168.0
21 7321632.0
22 7313808.0
23 7346064.0
24 7337280.0
25 7369344.0
26 7332192.0
27 7367712.0
28 7345536.0
29 7347504.0
---------------
41 번째를 합니다.
---------------
0 7316304.0
1 7328832.0
2 7335696.0
3 7350816.0
4 7316544.0
5 7330656.0
6 7336752.0
7 7336752.0
8 7333920.0
9 7325424.0
10 7340832.0
11 7341600.0
12 7342608.0
13 7353168.0
14 7336224.0
15 7296672.0
16 7355520.0
17 7336464.0
18 7326816.0
19 7331568.0
20 7330320.0
21 7343568.0
22 7340544.0
23 7350384.0
24 7329360.0
25 7353792.0
26 7371264.0
27 7356912.0
28 7343424.0
29 7357392.0
---------------
42 번째를 합니다.
---------------
0 7336368.0
1 7350480.0
2 7333920.0
3 7334688.0
4 7328352.0
5 7335120.0
6 7307856.0
7 7338768.0
8 7312224.0


---------------
60 번째를 합니다.
---------------
0 7389312.0
1 7385952.0
2 7381344.0
3 7360896.0
4 7397616.0
5 7391520.0
6 7362864.0
7 7389216.0
8 7383456.0
9 7386912.0
10 7385520.0
11 7399152.0
12 7385904.0
13 7383888.0
14 7365744.0
15 7387440.0
16 7381200.0
17 7387440.0
18 7397184.0
19 7388976.0
20 7378368.0
21 7385136.0
22 7386720.0
23 7397376.0
24 7398144.0
25 7403568.0
26 7407504.0
27 7396608.0
28 7405680.0
29 7395312.0
---------------
61 번째를 합니다.
---------------
0 7391184.0
1 7395456.0
2 7384272.0
3 7383552.0
4 7396800.0
5 7386528.0
6 7398384.0
7 7384320.0
8 7388880.0
9 7383936.0
10 7385520.0
11 7385520.0
12 7403664.0
13 7391328.0
14 7379376.0
15 7407504.0
16 7405344.0
17 7383360.0
18 7388448.0
19 7382448.0
20 7388064.0
21 7395456.0
22 7378512.0
23 7397424.0
24 7401072.0
25 7403808.0
26 7407024.0
27 7402416.0
28 7395984.0
29 7401312.0
---------------
62 번째를 합니다.
---------------
0 7391472.0
1 7392864.0
2 7385232.0
3 7385328.0
4 7405056.0
5 7395360.0
6 7382256.0
7 7380528.0
8 7389552.0


---------------
80 번째를 합니다.
---------------
0 7426608.0
1 7422912.0
2 7419936.0
3 7420032.0
4 7420704.0
5 7426224.0
6 7425696.0
7 7419024.0
8 7418784.0
9 7421760.0
10 7419216.0
11 7419024.0
12 7426944.0
13 7428576.0
14 7423728.0
15 7422864.0
16 7418688.0
17 7422768.0
18 7421088.0
19 7417680.0
20 7421568.0
21 7420176.0
22 7420176.0
23 7415856.0
24 7423728.0
25 7426944.0
26 7426080.0
27 7427808.0
28 7425408.0
29 7427568.0
---------------
81 번째를 합니다.
---------------
0 7418832.0
1 7427376.0
2 7426800.0
3 7418688.0
4 7431600.0
5 7423056.0
6 7416528.0
7 7426848.0
8 7419696.0
9 7424784.0
10 7424064.0
11 7422960.0
12 7426176.0
13 7424160.0
14 7420704.0
15 7423056.0
16 7421616.0
17 7422048.0
18 7426272.0
19 7420368.0
20 7422192.0
21 7421136.0
22 7422624.0
23 7423920.0
24 7427712.0
25 7427808.0
26 7427472.0
27 7427904.0
28 7425456.0
29 7428816.0
---------------
82 번째를 합니다.
---------------
0 7424064.0
1 7424832.0
2 7426128.0
3 7424208.0
4 7419312.0
5 7420656.0
6 7429152.0
7 7424544.0
8 7427184.0


...

---------------
200 번째를 합니다.
---------------
0 7507776.0
1 7504944.0
2 7507920.0
3 7507920.0
4 7508160.0
5 7503312.0
6 7509264.0
7 7501776.0
8 7507680.0
9 7503744.0
10 7504944.0
11 7503312.0
12 7504848.0
13 7502976.0
14 7504656.0
15 7505328.0
16 7507920.0
17 7507920.0
18 7507344.0
19 7507920.0
20 7507920.0
21 7507920.0
22 7505040.0
23 7502544.0
24 7503312.0
25 7509456.0
26 7509360.0
27 7508208.0
28 7508400.0
29 7508208.0
---------------
201 번째를 합니다.
---------------
0 7504944.0
1 7508112.0
2 7503504.0
3 7507920.0
4 7507728.0
5 7505520.0
6 7507824.0
7 7508496.0
8 7507920.0
9 7508208.0
10 7504992.0
11 7503984.0
12 7509264.0
13 7508208.0
14 7503408.0
15 7505040.0
16 7508016.0
17 7504848.0
18 7509168.0
19 7508016.0
20 7506768.0
21 7508016.0
22 7508208.0
23 7506912.0
24 7504944.0
25 7506288.0
26 7511040.0
27 7507536.0
28 7508688.0
29 7507920.0
---------------
202 번째를 합니다.
---------------
0 7504944.0
1 7506480.0
2 7507920.0
3 7508496.0
4 7510224.0
5 7504752.0
6 7508016.0
7 7508016.0
8 7504656

---------------
220 번째를 합니다.
---------------
0 7513584.0
1 7516464.0
2 7513152.0
3 7513296.0
4 7519824.0
5 7511952.0
6 7516656.0
7 7514016.0
8 7517088.0
9 7519536.0
10 7517088.0
11 7513536.0
12 7512480.0
13 7509600.0
14 7518864.0
15 7513344.0
16 7515264.0
17 7515408.0
18 7511136.0
19 7509456.0
20 7512048.0
21 7512864.0
22 7513488.0
23 7520064.0
24 7512240.0
25 7518384.0
26 7519776.0
27 7519536.0
28 7516704.0
29 7519824.0
---------------
221 번째를 합니다.
---------------
0 7510992.0
1 7515456.0
2 7509936.0
3 7512768.0
4 7512048.0
5 7512048.0
6 7516176.0
7 7510464.0
8 7515264.0
9 7514064.0
10 7515216.0
11 7519632.0
12 7514352.0
13 7513920.0
14 7517280.0
15 7510896.0
16 7515216.0
17 7513392.0
18 7515648.0
19 7513920.0
20 7513296.0
21 7518912.0
22 7519488.0
23 7513536.0
24 7516368.0
25 7520352.0
26 7519728.0
27 7519920.0
28 7519536.0
29 7519776.0
---------------
222 번째를 합니다.
---------------
0 7513344.0
1 7512000.0
2 7514112.0
3 7509744.0
4 7520592.0
5 7515216.0
6 7512288.0
7 7513824.0
8 7519344

---------------
240 번째를 합니다.
---------------
0 7522944.0
1 7521552.0
2 7526448.0
3 7528128.0
4 7524624.0
5 7528704.0
6 7522032.0
7 7527696.0
8 7520688.0
9 7522512.0
10 7525008.0
11 7527696.0
12 7527216.0
13 7528368.0
14 7516848.0
15 7527984.0
16 7525824.0
17 7525488.0
18 7523568.0
19 7528176.0
20 7527888.0
21 7517136.0
22 7521936.0
23 7527216.0
24 7527792.0
25 7529808.0
26 7528080.0
27 7528464.0
28 7528656.0
29 7527696.0
---------------
241 번째를 합니다.
---------------
0 7523376.0
1 7522080.0
2 7529232.0
3 7528080.0
4 7526496.0
5 7527696.0
6 7522416.0
7 7523664.0
8 7524528.0
9 7526736.0
10 7523568.0
11 7526352.0
12 7528368.0
13 7516848.0
14 7527888.0
15 7527456.0
16 7525584.0
17 7519920.0
18 7528560.0
19 7523184.0
20 7528176.0
21 7527888.0
22 7518384.0
23 7526256.0
24 7528272.0
25 7528752.0
26 7529328.0
27 7528800.0
28 7530096.0
29 7528272.0
---------------
242 번째를 합니다.
---------------
0 7522896.0
1 7523184.0
2 7526544.0
3 7526544.0
4 7521936.0
5 7523856.0
6 7527696.0
7 7530096.0
8 7528704

---------------
260 번째를 합니다.
---------------
0 7536720.0
1 7532880.0
2 7540464.0
3 7538352.0
4 7536912.0
5 7535664.0
6 7539360.0
7 7542672.0
8 7540176.0
9 7542912.0
10 7535568.0
11 7534416.0
12 7535856.0
13 7537296.0
14 7536240.0
15 7536624.0
16 7537584.0
17 7539408.0
18 7538064.0
19 7534896.0
20 7538064.0
21 7537776.0
22 7537488.0
23 7536816.0
24 7541424.0
25 7539504.0
26 7542384.0
27 7541760.0
28 7541424.0
29 7539984.0
---------------
261 번째를 합니다.
---------------
0 7540560.0
1 7542288.0
2 7538256.0
3 7538448.0
4 7536912.0
5 7536240.0
6 7536816.0
7 7537392.0
8 7535664.0
9 7535760.0
10 7538928.0
11 7538736.0
12 7538736.0
13 7535376.0
14 7537920.0
15 7539792.0
16 7537968.0
17 7535424.0
18 7534800.0
19 7536624.0
20 7538352.0
21 7535472.0
22 7540320.0
23 7542816.0
24 7541088.0
25 7542048.0
26 7542768.0
27 7542288.0
28 7540176.0
29 7541472.0
---------------
262 번째를 합니다.
---------------
0 7541232.0
1 7541616.0
2 7541328.0
3 7536192.0
4 7542576.0
5 7536432.0
6 7541520.0
7 7535712.0
8 7543440

---------------
280 번째를 합니다.
---------------
0 7555488.0
1 7552992.0
2 7555488.0
3 7553856.0
4 7555104.0
5 7555536.0
6 7549056.0
7 7552464.0
8 7543824.0
9 7554384.0
10 7551264.0
11 7548384.0
12 7549968.0
13 7550112.0
14 7551600.0
15 7547280.0
16 7547568.0
17 7552272.0
18 7554432.0
19 7554960.0
20 7555728.0
21 7554528.0
22 7553184.0
23 7555152.0
24 7552416.0
25 7553184.0
26 7555872.0
27 7555872.0
28 7554192.0
29 7555920.0
---------------
281 번째를 합니다.
---------------
0 7556064.0
1 7555488.0
2 7553184.0
3 7555872.0
4 7555344.0
5 7554336.0
6 7554720.0
7 7554768.0
8 7555920.0
9 7554912.0
10 7556256.0
11 7551648.0
12 7555488.0
13 7554528.0
14 7554768.0
15 7554720.0
16 7553232.0
17 7554336.0
18 7555872.0
19 7555536.0
20 7555536.0
21 7555536.0
22 7552224.0
23 7549344.0
24 7547088.0
25 7556400.0
26 7552800.0
27 7556064.0
28 7555728.0
29 7555728.0
---------------
282 번째를 합니다.
---------------
0 7555536.0
1 7546896.0
2 7556112.0
3 7555488.0
4 7552608.0
5 7556640.0
6 7550688.0
7 7555296.0
8 7556256

In [12]:
test_list

[[[[1, 1, 0, 0, 1, 1, 0, 1],
   [1, 0, 1, 0, 0, 1, 1, 0],
   [1, 0, 0, 0, 1, 1, 0, 1],
   [0, 0, 1, 0, 1, 0, 0, 0],
   [1, 0, 1, 1, 1, 1, 0, 0],
   [0, 1, 1, 0, 0, 1, 0, 0],
   [1, 0, 0, 0, 1, 0, 0, 1],
   [1, 1, 0, 1, 0, 0, 1, 1],
   [1, 1, 1, 0, 0, 0, 0, 1],
   [0, 0, 1, 1, 1, 1, 0, 0],
   [0, 1, 1, 1, 0, 1, 0, 0]],
  [[1, 1, 0, 0, 1, 1, 1, 0],
   [0, 1, 1, 1, 1, 1, 0, 1],
   [0, 0, 1, 1, 1, 0, 0, 0],
   [1, 0, 0, 0, 0, 1, 0, 1],
   [0, 1, 1, 0, 0, 1, 1, 1],
   [1, 1, 0, 0, 0, 1, 0, 1],
   [1, 0, 1, 0, 0, 0, 1, 0],
   [1, 0, 0, 0, 0, 0, 0, 0],
   [0, 0, 1, 0, 0, 0, 0, 1],
   [0, 1, 1, 1, 0, 1, 0, 0],
   [1, 0, 1, 0, 1, 1, 1, 0]],
  [[1, 0, 1, 0, 0, 0, 1, 1],
   [0, 1, 1, 1, 1, 1, 1, 0],
   [1, 1, 1, 0, 1, 1, 1, 0],
   [1, 0, 1, 0, 0, 0, 0, 1],
   [1, 1, 1, 1, 1, 0, 1, 1],
   [1, 0, 1, 0, 1, 1, 0, 0],
   [0, 1, 1, 1, 1, 1, 0, 0],
   [1, 0, 0, 1, 1, 0, 0, 1],
   [1, 1, 0, 1, 0, 1, 0, 1],
   [1, 0, 1, 0, 0, 1, 1, 1],
   [0, 1, 1, 0, 0, 0, 0, 1]],
  [[1, 0, 0, 0, 0, 0, 1, 0],
   [0, 0, 1

In [51]:
if speedlist[1][1][1][1] == 0:
    print("!!")
else:
    print("??")

??


In [ ]:
test_list2 = []

test_list2 = Evolution(test_list, 0)

In [37]:
Www = [[[0 for i in range(2)]for j in range(2)]for k in range(2)]
    # 교차함수 및 변이함수 (자식만들기)
    
print(Www)
for i in range(2):
    for j in range(2):
        for k in range(2):
            Www[i][j][k] = [43,43,0,2,2]
            
Www

[[[0, 0], [0, 0]], [[0, 0], [0, 0]]]


[[[[43, 43, 0, 2, 2], [43, 43, 0, 2, 2]],
  [[43, 43, 0, 2, 2], [43, 43, 0, 2, 2]]],
 [[[43, 43, 0, 2, 2], [43, 43, 0, 2, 2]],
  [[43, 43, 0, 2, 2], [43, 43, 0, 2, 2]]]]